#### Setup

In [ ]:
from specific import *

### Specify the experiments to compare

In [ ]:
experiments = ("15_most_important", "no_temporal_shifts")
assert len(experiments) == 2

### Load data

In [ ]:
experiment_data = load_experiment_data(experiments)

### Check that the masks are aligned

In [ ]:
comp_masks = [experiment_data[experiment]["master_mask"] for experiment in experiments]
assert np.all(comp_masks[0] == comp_masks[1])

### Predict BA

In [ ]:
master_mask = experiment_data[experiments[0]]["master_mask"]
target_ba = get_masked_array(
    experiment_data[experiments[0]]["endog_data"].values, master_mask
)

preds_cache = SimpleCache(f"predictions_{'_'.join(experiments)}", cache_dir=CACHE_DIR)

# XXX:
# preds_cache.clear()

n_threads = get_ncpus()


@preds_cache
def get_predictions():
    predictions = {}

    for experiment in experiments:
        experiment_data[experiment]["model"].n_jobs = n_threads
        with parallel_backend("threading", n_jobs=n_threads):
            predictions[experiment] = get_masked_array(
                experiment_data[experiment]["model"].predict(
                    experiment_data[experiment]["exog_data"]
                ),
                master_mask,
            )
    return predictions


predictions = get_predictions()

### Calculate Diffs

In [ ]:
diffs = predictions[experiments[1]] - predictions[experiments[0]]

### Compare the predictions

In [ ]:
plt.hist(get_unmasked(diffs), bins=100)
plt.yscale("log")
plt.xscale("symlog", linthres=1e-6)

In [ ]:
plt.hist(np.abs(get_unmasked(diffs)), bins=2000)
plt.yscale("log")
plt.xscale("symlog", linthres=1e-6)

In [ ]:
model_name = f"comparison_{'_'.join(experiments)}"

# Plotting params.
figsize = (5.1, 2.8)
mpl.rcParams["figure.figsize"] = figsize
coast_linewidth = 0.3
date_str = "2010-01 to 2015-01"

In [ ]:
boundaries = [-1e-2, -1e-3, -1e-4, 0, 1e-4, 1e-3, 1e-2]

# Plotting predicted.
fig = cube_plotting(
    diffs,
    title=f"BA <No Temporal - Top 15>\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=True,
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_{model_name}", sub_directory="predictions")

In [ ]:
boundaries = [-1e-1, 0, 0.1 ** 0.5, 1, 1e1]

# Plotting predicted.
fig = cube_plotting(
    diffs / predictions[experiments[0]],
    title=f"BA <No Temporal - Top 15> / Top 15\n{date_str}",
    boundaries=boundaries,
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    colorbar_kwargs={"label": "1", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"rel_ba_{model_name}", sub_directory="predictions")

In [ ]:
plot_data = np.abs(diffs)
vmax = np.max(plot_data)

boundaries = [1e-5, 1e-4, 1e-3, 1e-2, vmax]

# Plotting predicted.
fig = cube_plotting(
    np.abs(diffs),
    title=f"BA <|No Temporal - Top 15|>\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrRd",
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"ba_mean_abs_{model_name}", sub_directory="predictions")

In [ ]:
plot_data = np.abs(diffs)
vmax = np.max(plot_data)

boundaries = [0.1, 0.316, 1, 3.16, 10]

# Plotting predicted.
fig = cube_plotting(
    np.abs(diffs) / predictions[experiments[0]],
    title=f"BA <|No Temporal - Top 15|> / Top 15\n{date_str}",
    boundaries=boundaries,
    cmap="YlOrBr",
    colorbar_kwargs={"label": "1", "format": "%0.1e",},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"rel_ba_mean_abs_{model_name}", sub_directory="predictions"
)

### Load all feature data

In [ ]:
experiment = "fire_seasonality_paper"
all_experiment_data = load_experiment_data((experiment,))["fire_seasonality_paper"]

### Correlations between diffs and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
diff_data = get_unmasked(diffs)
all_features = all_experiment_data["exog_data"].copy()
all_features["Diffs"] = diff_data

In [ ]:
with figure_saver("diff_corr_plot"):
    corr_plot(shorten_columns(all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between rel. diffs and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
rel_diff_data = get_unmasked(diffs / predictions[experiments[0]])
rel_all_features = all_experiment_data["exog_data"].copy()
rel_all_features["Rel. Diffs"] = rel_diff_data

In [ ]:
with figure_saver("rel_diff_corr_plot"):
    corr_plot(shorten_columns(rel_all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between |diffs| and other variables - vritually no correlation between the diffs and any of the features

In [ ]:
abs_diff_data = np.abs(get_unmasked(diffs))
abs_all_features = all_experiment_data["exog_data"].copy()
abs_all_features["|Diffs|"] = abs_diff_data

In [ ]:
with figure_saver("abs_diff_corr_plot"):
    corr_plot(shorten_columns(abs_all_features), fig_kwargs={"figsize": (12, 8)})

### Correlations between rel. |diffs| and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
rel_abs_diff_data = np.abs(get_unmasked(diffs / predictions[experiments[0]]))
rel_abs_all_features = all_experiment_data["exog_data"].copy()
rel_abs_all_features["Rel. |Diffs|"] = rel_abs_diff_data

In [ ]:
with figure_saver("rel_abs_diff_corr_plot"):
    corr_plot(shorten_columns(rel_abs_all_features), fig_kwargs={"figsize": (12, 8)})